In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
dwadwa
# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 네이버 로그인 페이지로 이동
driver.get('https://nid.naver.com/nidlogin.login')

# 수동 로그인 안내 메시지
input("네이버에 로그인한 후 Enter 키를 눌러주세요...")

# 로그인 확인 대기
time.sleep(3)

# 네이버 카페 '수혜' 페이지로 이동
cafe_url = "https://cafe.naver.com/suhui"  
driver.get(cafe_url)

# 페이지 로드 대기
time.sleep(3)

# '이주의 인기글 모음' 카테고리 클릭
popular_post_xpath = '/html/body/div[3]/div/div[6]/div[1]/div[3]/div[2]/ul[2]/li[1]/a'
popular_category = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, popular_post_xpath)))
popular_category.click()

# iframe으로 전환
time.sleep(3)
driver.switch_to.frame("cafe_main")

# 충분히 스크롤을 내려 본문이 보이도록 설정
def scroll_down(driver, scroll_count=5):
    for _ in range(scroll_count):
        # 스크롤을 아래로 내림
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(1)  # 스크롤 후 로딩 시간 대기

# 스크롤을 5번 반복하여 본문까지 로드
scroll_down(driver, scroll_count=5)

# 게시글 데이터를 저장할 리스트
data = []

# 페이지에서 게시글 정보 수집 함수
def get_articles_from_page(driver):
    # 페이지가 로드될 때까지 대기
    time.sleep(2)
    
    # 현재 페이지의 게시글 크롤링
    articles = driver.find_elements(By.CSS_SELECTOR, '.article-board tbody tr')
    for article in articles:
        try:
            # 제목
            title_elem = article.find_elements(By.CLASS_NAME, 'td_article')
            title = title_elem[0].text.strip() if title_elem else "N/A"

            # 작성자
            writer_elem = article.find_elements(By.CLASS_NAME, 'td_name')
            writer = writer_elem[0].text.strip() if writer_elem else "N/A"

            # 작성일
            date_elem = article.find_elements(By.CLASS_NAME, 'td_date')
            date = date_elem[0].text.strip() if date_elem else "N/A"

            # 조회수
            views_elem = article.find_elements(By.CLASS_NAME, 'td_view')
            views = views_elem[0].text.strip().replace(",", "") if views_elem else "N/A"

            # 좋아요 (대소문자에 맞춰 'td_Likes' 사용)
            likes_elem = article.find_elements(By.CLASS_NAME, 'td_Likes')
            likes = likes_elem[0].text.strip().replace(",", "") if likes_elem else "N/A"
            
            # 게시글 정보를 리스트에 추가
            data.append({
                "제목": title,
                "작성자": writer,
                "작성일": date,
                "조회수": views,
                "좋아요": likes
            })
            
            # 수집된 게시글 정보를 출력
            print(f"제목: {title}, 작성자: {writer}, 작성일: {date}, 조회수: {views}, 좋아요: {likes}")
            
        except Exception as e:
            print("데이터 수집 중 오류:", e)

# 첫 페이지 크롤링
get_articles_from_page(driver)

# 다음 페이지로 이동하는 루프
while True:
    try:
        # "다음" 버튼 클릭
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='pgR']")))
        next_button.click()
        
        # 다음 페이지 게시글 크롤링
        scroll_down(driver, scroll_count=5)  # 다음 페이지에서도 충분히 스크롤
        get_articles_from_page(driver)
    except:
        print("더 이상 다음 페이지가 없습니다.")
        break

# 브라우저 종료
driver.quit()

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
df.to_csv("popular_articles.csv", index=False, encoding='utf-8-sig')
print("CSV 파일로 저장이 완료되었습니다. (popular_articles.csv)")

데이터 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".td_likes"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001027455dc cxxbridge1$str$ptr + 3653648
1   chromedriver                        0x000000010273de3c cxxbridge1$str$ptr + 3623024
2   chromedriver                        0x00000001021a8100 cxxbridge1$string$len + 88404
3   chromedriver                        0x00000001021ea41c cxxbridge1$string$len + 359536
4   chromedriver                        0x00000001021e0a58 cxxbridge1$string$len + 320172
5   chromedriver                        0x0000000102223ba8 cxxbridge1$string$len + 594940
6   chromedriver                        0x00000001021df0fc cxxbridge1$string$len + 313680
7   chromedriver                        0x00000001021dfd4c cxxb

In [8]:
import pandas as pd

df = pd.read_csv('popular_articles.csv')

In [9]:
df

,제목,작성자,작성일,조회수,좋아요
0,필독\n수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음 [3026],수만휘,2020.03.13.,84만,8079
1,28398566\n고대 국교 선배님들 감사합니다\n사진\n[22],ililililiiililli,2024.11.09.,"4,437",40
